# **Fine Tune GPT 2**

In [4]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 2.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [5]:
!pip install transformers_interpret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 851.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.9 MB/s eta 0:00:00


Fine Tune

In [ ]:
from transformers import Trainer, TrainingArguments, GPT2Tokenizer, GPT2LMHeadModel
from transformers import TextDataset, DataCollatorForLanguageModeling

In [ ]:
json_file_path = '/content/drive/MyDrive/Research_Project/Codes/JSON2GPT2/D3000/output.json'
val_json_file_path = '/content/drive/MyDrive/Research_Project/Codes/JSON2GPT2/D3000/val_output.json'

In [ ]:
def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size,
    )
    return dataset

In [ ]:
def load_data_collator(tokenizer, mlm=False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [ ]:
def compute_metrics(p):
    return {"perplexity": p.metrics["eval_loss"]}

In [ ]:
gradient_accumulation_steps = 4  # Accumulate gradients over multiple steps
per_device_train_batch_size = 2  # Adjust according to your GPU memory

In [ ]:
def train(train_file_path, eval_file_path, model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):

    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    train_dataset = load_dataset(train_file_path, tokenizer)
    eval_dataset = load_dataset(eval_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)

    tokenizer.save_pretrained(output_dir)

    model = GPT2LMHeadModel.from_pretrained(model_name)

    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=overwrite_output_dir,
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        num_train_epochs=num_train_epochs,
        evaluation_strategy="steps",  # Evaluate every save_steps steps
        save_strategy="steps",  # Save model every save_steps steps
        eval_steps=save_steps,
        save_steps=save_steps,
        logging_steps=save_steps,
        save_total_limit=2,
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    trainer.save_model()

In [ ]:
train_file_path = json_file_path
eval_file_path = val_json_file_path
model_name = 'gpt2'  # here we can use a bigger model like gpt2-medium, gpt2-large etc
output_dir = '/content/drive/MyDrive/Research_Project/Codes/Result2GPT2/D3000'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 12
save_steps = 5000

In [ ]:
train(
    train_file_path=train_file_path,
    eval_file_path=eval_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)


### **Get Model Predictions**

In [ ]:
# Load the fine-tuned model
model_name = '/content/drive/MyDrive/Research_Project/Codes/Result2GPT2/B8E3'
model = GPT2LMHeadModel.from_pretrained(model_name, local_files_only=True)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Input question
input_question = "A farmer has 120 acres of land. He plants corn on 3/4 of the land and soybeans on the rest. How many more acres of corn than soybeans does he plant?"
# Encode the input question
input_ids = tokenizer.encode(f"[Q] {input_question}", return_tensors="pt")

# Generate the equation
output = model.generate(input_ids, max_length=500, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0)
generated_equation = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_equation)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[Q] A farmer has 120 acres of land. He plants corn on 3/4 of the land and soybeans on the rest. How many more acres of corn than soybeans does he plant?",
   "[E] Let's denote the number of acres planted as \\( x \\) and the total amount he has to pay for all the corn and corn seeds. According to the information given:\n\n\\[ x = \\frac{120}{3.4} \\times 3x = 120 + 120 = 180 \\]\na = 90 \\text{ acres} \u00f7 90 = 360 \\nx \\cdot 360 = 0.5 \\)\na \\approx \\sqrt{180}{360} = 1.25 \\\nn = 2.75 \\}\nt = 3.375 \\% of \\[ \\left(x - 360)}{2.875} + \\right)(x + 180) = -1.125 \\\\\nz = 4.625 \\^2 + -0.05 \\nz \\u03c0 = 5.075 \\)\ny = 6.025 \\pi = 9.975 \\nn \\nd = 7.325 \\).\nb = (6.275 - -5) + (7.475 -6) \\nr = 8.825 \\nb \\nw = 10.895 \\).", (quoted from the second part of this article)\n- The farmer planted a total of 180 acres in the first year, so he paid a certain amount per acre for each acre planted. Since he planted 2/3 times as many corn as corn, we can set up the following equation 